# Variational Autoencoders for Colaborative Filtering on Movielens

In [1]:
import sys
import os
import numpy as np
import pandas as pd
import papermill as pm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import tensorflow as tf
import keras

from recommenders.utils.timer import Timer
from recommenders.datasets import movielens
from recommenders.datasets.split_utils import min_rating_filter_pandas
from recommenders.datasets.python_splitters import numpy_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k

from recommenders.datasets.sparse import AffinityMatrix
from recommenders.utils.python_utils import binarize
from recommenders.models.vae.multinomial_vae import Mult_VAE

from tempfile import TemporaryDirectory

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))
print("Keras version: {}".format(keras.__version__))

System version: 3.7.13 (default, Mar 29 2022, 02:18:16) 
[GCC 7.5.0]
Pandas version: 1.3.5
Tensorflow version: 2.7.3
Keras version: 2.7.0


In [4]:
# top k items to recommend
TOP_K = 100

# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '1m'

# Model parameters
HELDOUT_USERS = 600 # CHANGE FOR DIFFERENT DATASIZE
INTERMEDIATE_DIM = 200
LATENT_DIM = 70
EPOCHS = 400
BATCH_SIZE = 100

# temporary Path to save the optimal model's weights
tmp_dir = TemporaryDirectory()
WEIGHTS_PATH = os.path.join(tmp_dir.name, "mvae_weights.hdf5")

SEED = 98765

## 1. Multi-VAE algorithm
__Notations__: The notation used is described below.

$u \in \{1,\dots,U\}$ 는 유저 인덱스, $i \in \{1,\dots,I\}$ 는 아이템 인덱스이다. 유저-아이템 상호작용 행렬은 클릭에 대한 행렬 $\mathbf{X} \in \mathbb{N}^{U\times I}$ 이고, 모델의 입력이다. $\mathbf{x}_u =[X_{u1},\dots,X_{uI}]^\top \in \mathbb{N}^I$ 유저 *u*의 각 아이템에 대한 클릭 횟수를 나타내는 BoW 벡터이다. 클릭 행렬은 이진 행렬이다. 일반적인 카운트 데이터로 확장하기 쉽다.

__Multi-VAE Model__: 

각 유저 *u*에 대해, 표준 가우시안 사전확률로부터 $K$ 차원의 잠재 표현 $\mathbf{z}_u$ 를 샘플링하는 것으로 모델이 시작한다. $\mathbf{z}_u$ 는 $I$의 아이템들에 대해 확률 분포를 만들기 위해 비선형 함수 $f_\theta (\cdot) \in \mathbb{R}^I$ 를 통해 변형된다. 클릭 히스토리 $\mathbf{x}_u$ 에서, $\pi (\mathbf{z}_u)$ 에 대한 가정은 다음과 같다.
$$
\mathbf{z}_u \sim \mathcal{N}(0, \mathbf{I}_K),\\
\pi(\mathbf{z}_u) = softmax\{f_\theta (\mathbf{z}_u\},\\
\mathbf{x}_u \sim \mathrm{Mult}(N_u, \pi(\mathbf{z}_u))
$$

$\mathbf{z}_u$ 는 가우시안으로 가정되는 근사 사후 확률 $q_\phi (\mathbf{z}_u | \mathbf{x}_u )$ 로부터 샘플링되어야 한다. 경사도를 계산하기 위해, reparametrization trick을 사용하고 $\mathbf{z}_u$ 는 아래 공식으로 게산된다. 
$$
\mathbf{z}_u = \mathbf\mu(x_u)+\mathbf\sigma(x_u) \cdot \mathbf\epsilon
$$

where $\mathbf\epsilon \sim \mathcal{N}(0, \mathbf{I})$ and $ \mathbf\mu(x_u), \sigma(x_u)$ are calculated in encoder.


한 명이 유저 $u$ 를 위한 Multi-VAE의 목표는 :

$$
\mathcal{L}_u(\theta, \phi) = \mathbb{E}_{q_\phi(z_u | x_u)}[\log p_\theta(x_u | z_u)] - \beta \cdot KL(q_\phi(z_u | x_u) \| p(z_u))
$$

where $q_\phi$ is the approximating variational distribution (inference model/ encoder), and  $p_\theta$ refers to generative model/decoder. The first term is the log-likelohood and the second term is the  Kullback-Leibler divergence term.

첫번째 항을 다룰 때, 원 논문에 제시된 다항의 log-우도 공식을 사용한다.

$$\log p_\theta(\mathbf{x}_u | \mathbf{z}_u) = \sum_{i} \mathbf{x}_{ui}\log \mathbf{\pi}_i (z_u) $$

논문의 저자들은 직관적으로 내재적인 피드백 모델링에 더 좋은 다항 우도를 사용한다. 그리고 그들의 연구에서 증명된 것처럼 전통적으로 더 유명한 로지스틱이나 가우시안 우도보다 더 좋은 성능을 보여준다. 다항 우도를 사용함으로써 문제를 멀티 클래스 분류로 다룰 수 있다. 이 우도는 $x_u$ 의 0이 아닌 항목에 확률 밀도를 적용한 모델을 보상한다. 하지만 모델은 한정된 확률 밀도를 갖고 있다. 아이템들은 한정된 확률 밀도 안에서 경쟁한다. 모델은 더 클릭될만한 아이템에게 더 많은 확률 밀도를 할당한다.

또한, $\mathbf KL-divergence$ 는 정규화 텀으로 다뤄진다.[[Higgins et al, 2016]](https://openreview.net/pdf?id=Sy2fzU9gl), [[Burgess et al, 2018]](https://arxiv.org/pdf/1804.03599.pdf) : 


1. $\mathbf \beta = 1$ 이면 원래의 VAE 공식에 대응된다.[[Kingma et al, 2013]](https://arxiv.org/pdf/1312.6114.pdf)

2. $\mathbf \beta > 1$ 이라고 하면, 원래의 VAE 공식보다 잠재 보틀넥에 더 강한 제약을 준다. These constraints limit the capacity of $\mathbf z$, which, combined with the pressure to maximise the log likelihood of the training data $\mathbf x$ under the model, should encourage the model to learn the most efficient representation of the data.

3. Setting $\mathbf \beta<1$. 추천 시스템에서의 목표는 좋은 추천을 하는 것과 입력값이 초기의 입력값과 최대한 가깝도록 재구성하지 않는 것이다. 다른 말로 하면 로그 우도를 최대화하는 것이다. 만약 β<1로 설정하면, 사전 확률의 영향을 약화시키게 된다.  


As a result, for the reasons explained above in 3., the KL-divergence is multiplied by the parameter $\beta \in [0,1]$.

__Selecting β:__ 원 논문에 제시된 것처럼 $\mathbf \beta$ 를 선택하기 위해 간단한 경험적인 방법이 사용된다. 훈련 과정은 $\mathbf \beta = 0$에서 시작하고, 점차 1까지 증가한다. $\mathbf KL$ 텀은 $\mathbf \theta , \phi $ 에 대한 많은 그래디언트 업데이트에 걸쳐 서서히 어닐링된다. 최적의 $\mathbf \beta$ 는 모델의 성능이 정점을 찍었을 때 기록된다. 그러면 최적의 $\mathbf \beta$ 를 이용해 모델이 같은 어닐링 과정을 이용해 재학습되는데, $\mathbf \beta$ 는 이전 단계에서 찾아진 최적값에 도달했을 때 증가를 멈춘다.

이 방법은 잘 먹히고, 서로 다른 값의 $\mathbf \beta$ 를 갖는 여러 모델을 학습시킬 필요가 없다는 이점이 있다. [[Bowman et al, 2015]](https://arxiv.org/pdf/1511.06349.pdf)

## 2. Keras implementation of Multi-VAE
모델의 구현을 위해 Keras 패키지가 사용되었다.  Movielens 데이터는 클릭 행렬로 이진화되고, heldout 유저들 데이터에 기반해 검증된다.

## 3. Data Preparation
## 3.1 Load and split data
데이터는 train / validation / test 셋으로 나눠진다.
- 모든 유니크한 유저들은 training 유저와 heldout 유저로 나뉜다.
- 위의 유저들의 목록들을 이용해 해당하는 training 데이터와 heldout 데이터가 얻어지고, 클릭 행렬로 변환된다.
- 모델은 training 유저들의 전체 클릭 히스토리를 이용해 학습된다.
- 평가를 위해서 모델에 대한 필요한 user-level 표현을 학습하기 위해 heldout 데이터(validation, test)의 일부 클릭 히스토리가 사용된다. 그러면 평가 메트릭이 모델이 heldout 데이터의 unseen 클릭 데이터의 남은 부분을 얼마나 잘 랭킹시키는지를 기준으로 계산된다.

In [11]:
# Load data
df = movielens.load_pandas_df(size=MOVIELENS_DATA_SIZE,
                             header=['userID', 'itemID', 'rating', 'timestamp'])
df.to_csv('Movielens_1m.csv', index=False)

100%|█████████████████████████████████████| 5.78k/5.78k [00:03<00:00, 1.72kKB/s]


In [19]:
data = pd.read_csv('Movielens_1m.csv')
data.head()

,userID,itemID,rating,timestamp
0,1,1193,5.0,978300760
1,1,661,3.0,978302109
2,1,914,3.0,978301968
3,1,3408,4.0,978300275
4,1,2355,5.0,978824291


### Data Filtering
아래 규칙을 지켜 데이터를 필터링한다.
- 평점 3.5 이하의 데이터는 필터링한다. 유저로부터 3.5 이하의 평점을 받은 영화는 마지막 클릭 행렬에 포함되지 않는다. 이 필터링을 하지 않으면 최종 클릭 행렬은 엄청 희소할 것이다.
- 영화를 5개 미만으로 클릭한 유저는 필터링된다.
- 클릭되지 않은 영화는 필터링된다.

In [20]:
# Binarize the data
data_preferred = data[data['rating'] > 3.5]
print(data_preferred.shape)
data_low_rating = data[data['rating'] <= 3.5]

(575281, 4)


In [26]:
# users who clicked on at least 5 movies
data = min_rating_filter_pandas(data_preferred, min_rating=5, filter_by='user')
# movies that were clicked on by at least 1 user
data = min_rating_filter_pandas(data, min_rating=1, filter_by='item')

In [33]:
# obtain both usercnt and itemcnt after filtering
usercnt = data[['userID']].groupby('userID', as_index=False).size()
itemcnt = data[['itemID']].groupby('itemID', as_index=False).size()

# compute sparsity after filtering
sparsity = data.shape[0] / (usercnt.shape[0]*itemcnt.shape[0])
print("After filtering, there are %d watching events from %d users and %d movies (sparsity: %.3f%%)" % 
      (data.shape[0], usercnt.shape[0], itemcnt.shape[0], sparsity * 100))

After filtering, there are 575272 watching events from 6034 users and 3533 movies (sparsity: 2.699%)


### Split Data
검증 셋에 600명의 유저, 테스트 셋에 600명의 유저, 훈련 셋에 나머지 유저를 할당   
훈련 셋 유저들의 클릭 히스토리를 이용해 모델을 훈련시키기 때문에 검증 셋과 테스트 셋에 있는 영화들이 훈련 셋에 있는 영화들이어야 한다.

In [36]:
unique_users = sorted(data.userID.unique())
np.random.seed(SEED)
unique_users = np.random.permutation(unique_users)

n_users = len(unique_users)
train_users = unique_users[:(n_users - HELDOUT_USERS*2)]
val_users = unique_users[(n_users - HELDOUT_USERS*2):(n_users-HELDOUT_USERS)]
test_users = unique_users[(n_users-HELDOUT_USERS):]
print('Number of unique users :', n_users)
print('Number of training users :', len(train_users))
print('Number of validation users :', len(val_users))
print('Number of test users :', len(test_users))

Number of unique users : 6034
Number of training users : 4834
Number of validation users : 600
Number of test users : 600


In [37]:
train_set = data.loc[data['userID'].isin(train_users)]
val_set = data.loc[data['userID'].isin(val_users)]
test_set = data.loc[data['userID'].isin(test_users)]
print("Number of training observations: ", train_set.shape[0])
print("Number of validation observations: ", val_set.shape[0])
print("Number of test observations: ", test_set.shape[0])

Number of training observations:  462827
Number of validation observations:  57388
Number of test observations:  55057


In [40]:
# Obtain list of unique movies used in training set
unique_train_items = pd.unique(train_set['itemID'])
print("Number of unique movies that rated in training set", unique_train_items.size)

# Keep only movies that used in training set
val_set = val_set.loc[val_set['itemID'].isin(unique_train_items)]
print("Number of validation observations after filtering: ", val_set.shape[0])
test_set = test_set.loc[test_set['itemID'].isin(unique_train_items)]
print("Number of test observations after filtering: ", test_set.shape[0])

Number of unique movies that rated in training set 3497
Number of validation observations after filtering:  57374
Number of test observations after filtering:  55027


## 3.2 Click matrix generation
오직 0과 1로만 이루어진 클릭 행렬이 모델의 입력이 된다. 이 때 각 행은 유저를, 각 열은 영화를 표현한다. 결국 클릭 행렬은 유저의 선호도를 내포하고, 별로 좋아하지 않거나 보지 않은 영화는 0, 좋아한 영화는 1로 표시한다.   
Training set은 모든 training 유저들의 전체 기록을 포함하는 클릭 행렬이다. 하지만 validation set과 test set은 훈련과 테스트 부분으로 나눠져야 한다.   
`val_tr`은 유저마다 클릭 행렬에서 1로 표시된 영화의 75%를 포함한다. 나머지 25%는 `val_te`에 포함된다. test set에도 같은 규칙이 적용된다. `val_tr`은 각 에폭의 마지막에 모델의 입력으로 주어진다. 각 유저에게 모델에 의해 추천된 영화는 `reconstructed_val_tr`에 저장된다. 각 에폭에서 모델의 성능을 검증하기 위해 `NDCG@k` metric을 이용해 `reconstructed_val_tr`를 `val_te`와 비교한다.   
최종 검증을 위해 `test_tr`과 `test_te`를 사용한다. 과정은 각 에폭에서의 검증과 같다.

In [45]:
# Instantiate the sparse matrix generation
am_train = AffinityMatrix(df=train_set, items_list=unique_train_items)
am_val = AffinityMatrix(df=val_set, items_list=unique_train_items)
am_test = AffinityMatrix(df=test_set, items_list=unique_train_items)

# Obtain the sparse matrix
train_data, a, b = am_train.gen_affinity_matrix()
val_data, val_map_users, val_map_items = am_val.gen_affinity_matrix()
test_data, test_map_users, test_map_items = am_test.gen_affinity_matrix()
print(train_data.shape, val_data.shape, test_data.shape)

(4834, 3497) (600, 3497) (600, 3497)


In [46]:
train_data

array([[5., 4., 5., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 5., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.]])

In [47]:
# Split validation and test into training and testing parts
val_data_tr, val_data_te = numpy_stratified_split(val_data, ratio=0.75, seed=SEED)
test_data_tr, test_data_te = numpy_stratified_split(test_data, ratio=0.75, seed=SEED)

In [50]:
# Binarize data
train_data = binarize(a=train_data, threshold=3.5)
val_data = binarize(a=val_data, threshold=3.5)
test_data = binarize(a=test_data, threshold=3.5)
train_data

val_data_tr = binarize(a=val_data_tr, threshold=3.5)
# Save non-binary in separate object which will be used for calculating NDCG
val_data_te_ratings = val_data_te.copy()
val_data_te = binarize(a=val_data_te, threshold=3.5)

test_data_tr = binarize(a=test_data_tr, threshold=3.5)
# Save non-binary in separate object which will be used for calculating NDCG
test_data_te_ratings = test_data_te.copy()
test_data_te = binarize(a=test_data_te, threshold=3.5)